In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/IVSN

! pip install ml-collections
! pip3 install pickle5

In [18]:
import sys
import cv2
from google.colab.patches import cv2_imshow
import time
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import pickle
import pickle5 as pickle5
import random
from random import sample
import copy

import os
import shutil
from PIL import Image, ImageDraw

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
from torchvision import transforms

from utils import *
from SCEGRAM.SCEGRAM import SCEGRAM
sys.path.append("..")

In [3]:
context_dir = "../../SCEGRAM/01scenes/01object_present"
# target_dir = "../../SCEGRAM/02objects"
target_dir = "../../SCEGRAM/invariant_objects"
info_dir = "../../SCEGRAM/SCEGRAM_Database_scenes_objects.xlsx"
context_size, target_size = (320, 512), (128, 128)
dataset = SCEGRAM(info_dir, context_dir, target_dir, context_size, target_size)

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
# define IVSN model
class IVSN_sti(nn.Module):
  def __init__(self, model):
      super(IVSN_sti, self).__init__()
      self.features = nn.Sequential(*list(model.children())[0][:30])
      for param in self.features.parameters():
        param.requires_grad_ = False

  def forward(self, x):
      x = self.features(x)
      return x

class IVSN_tg(nn.Module):
  def __init__(self, model):
      super(IVSN_tg, self).__init__()
      self.features = nn.Sequential(*list(model.children())[0][:30])
      self.pool_layer = nn.AdaptiveMaxPool2d((1, 1))
      for param in self.features.parameters():
        param.requires_grad_ = False

  def forward(self, x):
      x = self.features(x)
      x = self.pool_layer(x)
      return x

from torch.nn.modules.conv import Conv2d
ConvSize, NumTemplates, Mylayer = 1, 512, 31
MMconv = Conv2d(NumTemplates, 1, kernel_size = (ConvSize, ConvSize), stride = (1, 1), padding = (1, 1))

In [5]:
model_vgg = models.vgg16(pretrained=True)
model_ivsn_sti = IVSN_sti(model_vgg)
model_ivsn_tg = IVSN_tg(model_vgg)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [6]:
with open("[SCEGRAM]bin_idxs.pkl", "rb") as tf:
    bin_info = pickle5.load(tf) 

In [7]:
num_pics, size, image_size = len(dataset), 48, (320, 512)
IVSN_CON_0_25, IVSN_CON_25_50 = [], []
IVSN_INCON_0_25, IVSN_INCON_25_50 = [], []
scanpath, attention_map = {}, {}
# index of selected images of first two bins
selected_imgs = bin_info['con_(0, 25]'].tolist() + bin_info['con_(25, 50]'].tolist() + bin_info['incon_(0, 25]'].tolist() + bin_info['incon_(25, 50]'].tolist()

model_ivsn_sti.eval()
model_ivsn_tg.eval()
with torch.no_grad():
    for id in trange(0, num_pics):
        if id not in selected_imgs:
            continue

        context_images, target_images, bbox, category = dataset[id]
        # get attention map from IVSN model
        context_ivsn = context_images.unsqueeze(0)
        target_ivsn = target_images.unsqueeze(0)
        cont_output_ivsn = model_ivsn_sti(context_ivsn)
        tg_output_ivsn = model_ivsn_tg(target_ivsn)
        MMconv.weight = torch.nn.Parameter(tg_output_ivsn)
        attention_IVSN = MMconv.forward(cont_output_ivsn)
        attention_IVSN = attention_IVSN.detach().squeeze(0)

        # calculate the target bounding box
        tg_loc = bbox_cordinates(bbox, image_size[1], image_size[0])

        # process IVSN attention map
        mask_IVSN = transforms.Resize(image_size)(attention_IVSN)
        mask_IVSN = torch.divide(mask_IVSN, mask_IVSN.max())

        # save the attention map
        attention_map[id] = (copy.deepcopy(mask_IVSN))

        IVSN_num, path = searchProcesswithPath(tg_loc, mask_IVSN, image_size, size)

        scanpath[id] = path

        if id in bin_info['con_(0, 25]'].tolist():
            IVSN_CON_0_25.append(IVSN_num)
        elif id in bin_info['con_(25, 50]'].tolist():
            IVSN_CON_25_50.append(IVSN_num)

        elif id in bin_info['incon_(0, 25]'].tolist():
            IVSN_INCON_0_25.append(IVSN_num)
        elif id in bin_info['incon_(25, 50]'].tolist():
            IVSN_INCON_25_50.append(IVSN_num)    

        print('IVSN_' + str(id) + ': ' + str(IVSN_num), end = '\t')

IVSN_CON_res = IVSN_CON_0_25 + IVSN_CON_25_50
IVSN_INCON_res = IVSN_INCON_0_25 + IVSN_INCON_25_50
IVSN_res = IVSN_CON_res + IVSN_INCON_res

  0%|          | 1/372 [00:06<39:40,  6.42s/it]

IVSN_0: 2	

  1%|          | 2/372 [00:09<27:16,  4.42s/it]

IVSN_1: 2	

  1%|          | 3/372 [00:11<21:07,  3.43s/it]

...

IVSN_364: 2	

 99%|█████████▉| 370/372 [07:03<00:01,  1.29it/s]

IVSN_369: 3	

100%|██████████| 372/372 [07:05<00:00,  1.14s/it]

IVSN_371: 9	

In [8]:
np.mean(IVSN_res), np.mean(IVSN_CON_res), np.mean(IVSN_INCON_res)

(7.652406417112299, 5.21875, 8.154838709677419)

In [16]:
def sampleIncon(incon_bin_result, con_bin_result, times):
    sample_times = times
    nums = len(con_bin_result)
    print(nums)
    res = np.array([0.0] * 25)
    
    for id in range(sample_times):
        temp = sample(incon_bin_result, nums)
        temp_accu = model_performance(temp, len(temp))
        res += np.array(temp_accu[:25])

    return (res/sample_times).tolist()

def balanced_accu(res_con, res_incon):
    res = []
    for i in range(25):
        res.append((res_con[i]+res_incon[i])/2)

    return res

In [9]:
len(IVSN_res) == len(IVSN_CON_res) + len(IVSN_INCON_res)

True

In [19]:
times = 100
IVSN_CON_accu = model_performance(IVSN_CON_res, len(IVSN_CON_res))
IVSN_INCON_accu = sampleIncon(IVSN_INCON_res, IVSN_CON_res, times)
IVSN_accu = balanced_accu(IVSN_CON_accu, IVSN_INCON_accu)
IVSN_accu[:10]

32


[0.0,
 0.1759375,
 0.5426562500000001,
 0.60234375,
 0.67828125,
 0.7029687499999999,
 0.7390625,
 0.7646875,
 0.77640625,
 0.77890625]

In [22]:
IVSN_CON_accu[:10], IVSN_INCON_accu[:10]

([0, 0.25, 0.625, 0.6875, 0.75, 0.75, 0.78125, 0.8125, 0.8125, 0.8125],
 [0.0,
  0.101875,
  0.4603125,
  0.5171875,
  0.6065625,
  0.6559375,
  0.696875,
  0.716875,
  0.7403125,
  0.7453125])

In [21]:
IVSN_SCEGRAM_res = {}
IVSN_SCEGRAM_res['combined_accu'] = IVSN_accu
IVSN_SCEGRAM_res['con_accu'] = IVSN_CON_accu
IVSN_SCEGRAM_res['incon_accu'] = IVSN_INCON_accu
IVSN_SCEGRAM_res['con_[0,25)'] = IVSN_CON_0_25
IVSN_SCEGRAM_res['con_[25,50)'] = IVSN_CON_25_50
IVSN_SCEGRAM_res['incon_[0,25)'] = IVSN_INCON_0_25
IVSN_SCEGRAM_res['incon_[25,50)'] = IVSN_INCON_25_50
IVSN_SCEGRAM_res['scanpath'] = scanpath
IVSN_SCEGRAM_res['attention_map'] = attention_map

In [23]:
with open("../results/SCEGRAM/SCEGRAM(invariant_bin1_2)_IVSN_res.pkl", "wb") as tf:
    pickle.dump(IVSN_SCEGRAM_res, tf)